# 어댑터 로드


[[colab에서 열기]]

특정 주제의 이미지를 생성하거나 특정 스타일의 이미지를 생성하도록 확산 모델을 개인화하는 여러 [학습](../training/overview) 기법이 있습니다. 이러한 각 학습 방법은 다른 유형의 어댑터를 생성합니다. 일부 어댑터는 완전히 새로운 모델을 생성하는 반면, 다른 어댑터는 더 작은 임베딩 또는 가중치 집합만 수정합니다. 즉, 각 어댑터의 로드 프로세스도 다릅니다.

이 가이드에서는 드림부스, 텍스트 반전 및 LoRA 가중치를 로드하는 방법을 보여줍니다.

<Tip>

[Stable Diffusion Conceptualizer](https://huggingface.co/spaces/sd-concepts-library/stable-diffusion-conceptualizer), [LoRA the Explorer](https://huggingface.co/spaces/multimodalart/LoraTheExplorer) 및 [Diffusers Models Gallery](https://huggingface.co/spaces/huggingface-projects/diffusers-gallery)에서 사용할 체크포인트 및 임베딩을 자유롭게 찾아보십시오.

</Tip>

## 라이브러리 가져오기

In [ ]:
! pip install diffusers transformers accelerate peft

## 드림부스


[DreamBooth](https://dreambooth.github.io/)는 피사체의 이미지 몇 장만으로 *전체 확산 모델*을 미세 조정하여 새로운 스타일과 설정에서 해당 피사체의 이미지를 생성합니다. 이 방법은 모델이 피사체 이미지와 연관시키는 법을 배우는 프롬프트에서 특수 단어를 사용하여 작동합니다. 모든 학습 방법 중에서 DreamBooth는 전체 체크포인트 모델이기 때문에 가장 큰 파일 크기(일반적으로 몇 GB)를 생성합니다.

헤르제가 그린 단 10개의 이미지로 학습된 [herge_style](https://huggingface.co/sd-dreambooth-library/herge-style) 체크포인트를 로드하여 해당 스타일의 이미지를 생성해 보겠습니다. 작동하려면 프롬프트에 특수 단어 `herge_style`을 포함하여 체크포인트를 트리거해야 합니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained("sd-dreambooth-library/herge-style", torch_dtype=torch.float16).to("cuda")
prompt = "A cute herge_style brown bear eating a slice of pizza, stunning color scheme, masterpiece, illustration"
image = pipeline(prompt).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/load_dreambooth.png" />
</div>

## 텍스트 반전


[텍스트 반전](https://textual-inversion.github.io/)은 드림부스와 매우 유사하며 확산 모델을 개인화하여 몇 장의 이미지에서 특정 개념(스타일, 객체)을 생성할 수도 있습니다. 이 방법은 프롬프트에 특수 단어를 사용하여 제공한 이미지를 나타내는 새로운 임베딩을 학습하고 찾는 방식으로 작동합니다. 결과적으로 확산 모델 가중치는 동일하게 유지되고 학습 프로세스는 비교적 작은(몇 KB) 파일을 생성합니다.

텍스트 반전은 임베딩을 생성하므로 드림부스와 같이 자체적으로 사용할 수 없으며 다른 모델이 필요합니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")

이제 [`~loaders.TextualInversionLoaderMixin.load_textual_inversion`] 메서드를 사용하여 텍스트 반전 임베딩을 로드하고 일부 이미지를 생성할 수 있습니다. [sd-concepts-library/gta5-artwork](https://huggingface.co/sd-concepts-library/gta5-artwork) 임베딩을 로드하고 이를 트리거하려면 프롬프트에 특수 단어 `<gta5-artwork>`를 포함해야 합니다.

In [ ]:
pipeline.load_textual_inversion("sd-concepts-library/gta5-artwork")
prompt = "A cute brown bear eating a slice of pizza, stunning color scheme, masterpiece, illustration, <gta5-artwork> style"
image = pipeline(prompt).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/load_txt_embed.png" />
</div>

텍스트 반전은 흐릿한 이미지나 손가락이 추가된 손과 같이 바람직하지 않은 것을 생성하지 않도록 모델을 유도하는 *음수 임베딩*을 만드는 데에도 사용할 수 있습니다. 이는 프롬프트를 빠르게 개선하는 쉬운 방법이 될 수 있습니다. 또한 [`~loaders.TextualInversionLoaderMixin.load_textual_inversion`]을 사용하여 임베딩을 로드하지만 이번에는 두 가지 매개변수가 더 필요합니다.

- `weight_name`: 파일이 특정 이름으로 🤗 Diffusers 형식으로 저장되었거나 파일이 A1111 형식으로 저장된 경우 로드할 가중치 파일을 지정합니다.
- `token`: 프롬프트에서 임베딩을 트리거하는 데 사용할 특수 단어를 지정합니다.

[EvilEngine/easynegative](https://huggingface.co/EvilEngine/easynegative) 임베딩을 로드해 보겠습니다.

In [ ]:
pipeline.load_textual_inversion(
    "EvilEngine/easynegative", weight_name="easynegative.safetensors", token="EasyNegative"
)

이제 `token`을 사용하여 음수 임베딩으로 이미지를 생성할 수 있습니다.

In [ ]:
prompt = "A cute brown bear eating a slice of pizza, stunning color scheme, masterpiece, illustration, EasyNegative"
negative_prompt = "EasyNegative"

image = pipeline(prompt, negative_prompt=negative_prompt, num_inference_steps=50).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/load_neg_embed.png" />
</div>

## LoRA

[저순위 적응(LoRA)](https://huggingface.co/papers/2106.09685)는 빠르고 작은 파일 크기(수백 MB)를 생성하기 때문에 인기 있는 학습 기법입니다. 이 가이드의 다른 방법과 마찬가지로 LoRA는 모델이 몇 장의 이미지에서 새로운 스타일을 학습하도록 학습할 수 있습니다. 확산 모델에 새로운 가중치를 삽입한 다음 전체 모델 대신 새로운 가중치만 학습하는 방식으로 작동합니다. 이렇게 하면 LoRA를 더 빠르게 학습하고 더 쉽게 저장할 수 있습니다.

<Tip>

LoRA는 다른 학습 방법과 함께 사용할 수 있는 매우 일반적인 학습 기법입니다. 예를 들어, 드림부스와 LoRA로 모델을 학습하는 것이 일반적입니다. 또한 여러 LoRA를 로드하고 병합하여 새롭고 독특한 이미지를 만드는 것도 점점 일반화되고 있습니다. 병합은 이 로드 가이드의 범위를 벗어나므로 심층적인 [LoRA 병합](merge_loras) 가이드에서 자세히 알아볼 수 있습니다.

</Tip>

LoRA는 다른 모델과 함께 사용해야 합니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16).to("cuda")

그런 다음 [`~loaders.StableDiffusionLoraLoaderMixin.load_lora_weights`] 메서드를 사용하여 [ostris/super-cereal-sdxl-lora](https://huggingface.co/ostris/super-cereal-sdxl-lora) 가중치를 로드하고 리포지토리에서 가중치 파일 이름을 지정합니다.

In [ ]:
pipeline.load_lora_weights("ostris/super-cereal-sdxl-lora", weight_name="cereal_box_sdxl_v1.safetensors")
prompt = "bears, pizza bites"
image = pipeline(prompt).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/load_lora.png" />
</div>

[`~loaders.StableDiffusionLoraLoaderMixin.load_lora_weights`] 메서드는 UNet과 텍스트 인코더 모두에 LoRA 가중치를 로드합니다. 다음과 같은 경우를 처리할 수 있으므로 LoRA를 로드하는 데 선호되는 방법입니다.

- LoRA 가중치에 UNet과 텍스트 인코더에 대한 별도의 식별자가 없는 경우
- LoRA 가중치에 UNet과 텍스트 인코더에 대한 별도의 식별자가 있는 경우

그러나 UNet에만 LoRA 가중치를 로드해야 하는 경우 [`~loaders.UNet2DConditionLoadersMixin.load_attn_procs`] 메서드를 사용할 수 있습니다. [jbilcke-hf/sdxl-cinematic-1](https://huggingface.co/jbilcke-hf/sdxl-cinematic-1) LoRA를 로드해 보겠습니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16).to("cuda")
pipeline.unet.load_attn_procs("jbilcke-hf/sdxl-cinematic-1", weight_name="pytorch_lora_weights.safetensors")

# 프롬프트에 cnmt를 사용하여 LoRA를 트리거합니다.
prompt = "A cute cnmt eating a slice of pizza, stunning color scheme, masterpiece, illustration"
image = pipeline(prompt).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/load_attn_proc.png" />
</div>

LoRA 가중치를 언로드하려면 [`~loaders.StableDiffusionLoraLoaderMixin.unload_lora_weights`] 메서드를 사용하여 LoRA 가중치를 버리고 모델을 원래 가중치로 복원합니다.

In [ ]:
pipeline.unload_lora_weights()

### LoRA 가중치 척도 조정


[`~loaders.StableDiffusionLoraLoaderMixin.load_lora_weights`]와 [`~loaders.UNet2DConditionLoadersMixin.load_attn_procs`] 모두에 대해 `cross_attention_kwargs={"scale": 0.5}` 매개변수를 전달하여 사용할 LoRA 가중치의 양을 조정할 수 있습니다. 값 `0`은 기본 모델 가중치만 사용하는 것과 같고, 값 `1`은 완전히 미세 조정된 LoRA를 사용하는 것과 같습니다.

계층별로 사용되는 LoRA 가중치의 양을 보다 세밀하게 제어하려면 [`~loaders.StableDiffusionLoraLoaderMixin.set_adapters`]를 사용하고 각 계층의 가중치를 얼마나 조정할지 지정하는 사전을 전달할 수 있습니다.

In [ ]:
# 의사 코드
pipe = ... # 파이프라인 생성
pipe.load_lora_weights(..., adapter_name="my_adapter")
scales = {
    "text_encoder": 0.5,
    "text_encoder_2": 0.5,  # 파이프라인에 두 번째 텍스트 인코더가 있는 경우에만 사용 가능
    "unet": {
        "down": 0.9,  # 다운 파트의 모든 트랜스포머는 스케일 0.9를 사용합니다.
        # "mid"  # 이 예에서는 "mid"가 주어지지 않았으므로 미드 파트의 모든 트랜스포머는 기본 스케일 1.0을 사용합니다.
        "up": {
            "block_0": 0.6,  # 업 파트의 0번째 블록에 있는 모든 3개의 트랜스포머는 스케일 0.6을 사용합니다.
            "block_1": [0.4, 0.8, 1.0],  # 업 파트의 1번째 블록에 있는 3개의 트랜스포머는 각각 스케일 0.4, 0.8 및 1.0을 사용합니다.
        }
    }
}
pipe.set_adapters("my_adapter", scales)

이것은 여러 어댑터에서도 작동합니다. 자세한 내용은 [이 가이드](https://huggingface.co/docs/diffusers/tutorials/using_peft_for_inference#customize-adapters-strength)를 참조하십시오.

<Tip warning={true}>

현재 [`~loaders.StableDiffusionLoraLoaderMixin.set_adapters`]는 어텐션 가중치 조정만 지원합니다. LoRA에 다른 부분(예: resnet 또는 다운/업샘플러)이 있는 경우 스케일 1.0을 유지합니다.

</Tip>

### 코햐와 더라스트벤


커뮤니티의 다른 인기 있는 LoRA 트레이너로는 [코햐](https://github.com/kohya-ss/sd-scripts/)와 [더라스트벤](https://github.com/TheLastBen/fast-stable-diffusion)이 있습니다. 이러한 트레이너는 🤗 Diffusers에서 학습된 것과는 다른 LoRA 체크포인트를 생성하지만 동일한 방식으로 로드할 수 있습니다.

<hfoptions id="other-trainers">
<hfoption id="Kohya">

코햐 LoRA를 로드하려면 예시로 [Civitai](https://civitai.com/)에서 [Blueprintify SD XL 1.0](https://civitai.com/models/150986/blueprintify-sd-xl-10) 체크포인트를 다운로드해 보겠습니다.

In [ ]:
!wget https://civitai.com/api/download/models/168776 -O blueprintify-sd-xl-10.safetensors

[`~loaders.StableDiffusionLoraLoaderMixin.load_lora_weights`] 메서드를 사용하여 LoRA 체크포인트를 로드하고 `weight_name` 매개변수에 파일 이름을 지정합니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16).to("cuda")
pipeline.load_lora_weights("path/to/weights", weight_name="blueprintify-sd-xl-10.safetensors")

이미지 생성:

In [ ]:
# 프롬프트에 bl3uprint를 사용하여 LoRA를 트리거합니다.
prompt = "bl3uprint, a highly detailed blueprint of the eiffel tower, explaining how to build all parts, many txt, blueprint grid backdrop"
image = pipeline(prompt).images[0]
image

<Tip warning={true}>

🤗 Diffusers에서 Kohya LoRA를 사용하는 데에는 몇 가지 제한 사항이 있습니다.

- 이미지가 ComfyUI와 같은 UI에서 생성된 이미지와 다르게 보일 수 있으며, 그 이유는 여러 가지가 있으며 [여기](https://github.com/huggingface/diffusers/pull/4287/#issuecomment-1655110736)에서 설명합니다.
- [LyCORIS 체크포인트](https://github.com/KohakuBlueleaf/LyCORIS)는 완전히 지원되지 않습니다. [`~loaders.StableDiffusionLoraLoaderMixin.load_lora_weights`] 메서드는 LoRA 및 LoCon 모듈이 있는 LyCORIS 체크포인트를 로드하지만 Hada 및 LoKR은 지원되지 않습니다.

</Tip>

</hfoption>
<hfoption id="TheLastBen">

TheLastBen에서 체크포인트를 로드하는 것은 매우 유사합니다. 예를 들어 [TheLastBen/William_Eggleston_Style_SDXL](https://huggingface.co/TheLastBen/William_Eggleston_Style_SDXL) 체크포인트를 로드하려면 다음을 수행합니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16).to("cuda")
pipeline.load_lora_weights("TheLastBen/William_Eggleston_Style_SDXL", weight_name="wegg.safetensors")

# 프롬프트에 윌리엄 에글스턴을 사용하여 LoRA를 트리거합니다.
prompt = "a house by william eggleston, sunrays, beautiful, sunlight, sunrays, beautiful"
image = pipeline(prompt=prompt).images[0]
image

</hfoption>
</hfoptions>

## IP 어댑터


[IP-Adapter](https://ip-adapter.github.io/)는 모든 확산 모델에 대해 이미지 프롬프팅을 가능하게 하는 경량 어댑터입니다. 이 어댑터는 이미지와 텍스트 특징의 교차 어텐션 계층을 분리하여 작동합니다. 다른 모든 모델 구성 요소는 고정되고 UNet의 내장된 이미지 특징만 학습됩니다. 결과적으로 IP-Adapter 파일은 일반적으로 약 100MB에 불과합니다.

다양한 작업 및 특정 사용 사례에 대한 IP-Adapter 사용 방법에 대한 자세한 내용은 [IP-Adapter](../using-diffusers/ip_adapter) 가이드에서 확인할 수 있습니다.

> [!TIP]
> Diffusers는 현재 가장 인기 있는 일부 파이프라인에 대해서만 IP-Adapter를 지원합니다. 멋진 사용 사례가 있고 지원되지 않는 파이프라인과 IP-Adapter를 통합하고 싶다면 언제든지 기능 요청을 열어주세요!
> 공식 IP-Adapter 체크포인트는 [h94/IP-Adapter](https://huggingface.co/h94/IP-Adapter)에서 사용할 수 있습니다.

시작하려면 Stable Diffusion 체크포인트를 로드합니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch
from diffusers.utils import load_image

pipeline = AutoPipelineForText2Image.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")

그런 다음 IP-Adapter 가중치를 로드하고 [`~loaders.IPAdapterMixin.load_ip_adapter`] 메서드를 사용하여 파이프라인에 추가합니다.

In [ ]:
pipeline.load_ip_adapter("h94/IP-Adapter", subfolder="models", weight_name="ip-adapter_sd15.bin")

로드되면 이미지 및 텍스트 프롬프트를 사용하여 파이프라인을 사용하여 이미지 생성 프로세스를 안내할 수 있습니다.

In [ ]:
image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/load_neg_embed.png")
generator = torch.Generator(device="cpu").manual_seed(33)
images = pipeline(
    prompt='best quality, high quality, wearing sunglasses',
    ip_adapter_image=image,
    negative_prompt="monochrome, lowres, bad anatomy, worst quality, low quality",
    num_inference_steps=50,
    generator=generator,
).images[0]
images

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/YiYiXu/testing-images/resolve/main/ip-bear.png" />
</div>

### IP 어댑터 플러스


IP-Adapter는 이미지 특징을 생성하기 위해 이미지 인코더에 의존합니다. IP-Adapter 리포지토리에 `image_encoder` 하위 폴더가 포함되어 있으면 이미지 인코더가 자동으로 로드되어 파이프라인에 등록됩니다. 그렇지 않으면 [`~transformers.CLIPVisionModelWithProjection`] 모델을 사용하여 이미지 인코더를 명시적으로 로드하고 파이프라인에 전달해야 합니다.

ViT-H 이미지 인코더를 사용하는 *IP-Adapter Plus* 체크포인트의 경우입니다.

In [ ]:
from transformers import CLIPVisionModelWithProjection

image_encoder = CLIPVisionModelWithProjection.from_pretrained(
    "h94/IP-Adapter",
    subfolder="models/image_encoder",
    torch_dtype=torch.float16
)

pipeline = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    image_encoder=image_encoder,
    torch_dtype=torch.float16
).to("cuda")

pipeline.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name="ip-adapter-plus_sdxl_vit-h.safetensors")

### IP 어댑터 얼굴 ID 모델

IP-Adapter FaceID 모델은 CLIP 이미지 임베딩 대신 `insightface`에서 생성된 이미지 임베딩을 사용하는 실험적인 IP 어댑터입니다. 일부 모델은 ID 일관성을 향상시키기 위해 LoRA도 사용합니다.
이러한 모델을 사용하려면 `insightface`와 모든 요구 사항을 설치해야 합니다.

<Tip warning={true}>
InsightFace 사전 학습 모델은 비상업적 연구 목적으로 사용할 수 있으므로 IP-Adapter-FaceID 모델은 연구 목적으로만 출시되며 상업적 용도로는 사용할 수 없습니다.
</Tip>

In [ ]:
pipeline = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16
).to("cuda")

pipeline.load_ip_adapter("h94/IP-Adapter-FaceID", subfolder=None, weight_name="ip-adapter-faceid_sdxl.bin", image_encoder_folder=None)

두 가지 IP-Adapter FaceID Plus 모델 중 하나를 사용하려면 CLIP 이미지 인코더도 로드해야 합니다. 이 모델은 더 나은 사진 사실성을 얻기 위해 `insightface`와 CLIP 이미지 임베딩을 모두 사용하기 때문입니다.

In [ ]:
from transformers import CLIPVisionModelWithProjection

image_encoder = CLIPVisionModelWithProjection.from_pretrained(
    "laion/CLIP-ViT-H-14-laion2B-s32B-b79K",
    torch_dtype=torch.float16,
)

pipeline = AutoPipelineForText2Image.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5",
    image_encoder=image_encoder,
    torch_dtype=torch.float16
).to("cuda")

pipeline.load_ip_adapter("h94/IP-Adapter-FaceID", subfolder=None, weight_name="ip-adapter-faceid-plus_sd15.bin")